<a href="https://colab.research.google.com/github/rizkiar00/NaiveBayes/blob/master/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORT

In [158]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; 
from google.colab import files
from sklearn.model_selection import train_test_split
import math
import operator 
%matplotlib inline
!git clone https://github.com/rizkiar00/NaiveBayes

fatal: destination path 'NaiveBayes' already exists and is not an empty directory.


## PISAH TRAINING DAN TESTING

In [159]:
data = pd.read_csv('NaiveBayes/TrainsetTugas1ML.csv',skipinitialspace=True) #Masukkan Data dan Buat Data Training,Testing
databig = data.loc[data['income'] == '>50K']
datasmall = data.loc[data['income'] == '<=50K']
X_trainsmall, X_testsmall, y_trainsmall, y_testsmall = train_test_split(datasmall.drop('income',axis=1), datasmall['income'], test_size=0.025, random_state=42)
X_trainbig, X_testbig, y_trainbig, y_testbig = train_test_split(databig.drop('income',axis=1), databig['income'], test_size=0.5, random_state=42)
datatrain = pd.concat([X_trainsmall,X_trainbig], ignore_index=True)
datatest = pd.concat([X_testsmall,X_testbig], ignore_index=True)
y_train = pd.concat([y_trainsmall,y_trainbig], ignore_index=True)
y_text = pd.concat([y_testsmall,y_testbig], ignore_index=True)
# datatrainbig = datatrain.loc[datatrain['income'] == '>50K'].reset_index().drop(["index"], axis=1)
# datatrainsmall = datatrain.loc[datatrain['income'] == '<=50K'].reset_index().drop(["index"], axis=1)
datatrain = pd.concat([datatrain, y_train], axis=1)
datatest = pd.concat([datatest, y_text], axis=1)
datatest

,id,age,workclass,education,marital-status,occupation,relationship,hours-per-week,income
0,3019,adult,Private,Some-college,Never-married,Exec-managerial,Not-in-family,normal,<=50K
1,6045,adult,Self-emp-not-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,normal,>50K
2,27988,adult,Local-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,normal,>50K
3,13128,adult,Local-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,normal,>50K
4,29274,young,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,normal,>50K
5,25885,young,Self-emp-not-inc,Bachelors,Never-married,Prof-specialty,Not-in-family,normal,>50K
6,19644,adult,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,normal,>50K
7,6994,adult,Local-gov,HS-grad,Married-civ-spouse,Exec-managerial,Husband,normal,>50K
8,9847,young,Self-emp-not-inc,Some-college,Married-civ-spouse,Craft-repair,Husband,normal,>50K
9,24113,young,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,normal,>50K


## HITUNG PRIOR & LIKELIHOOD

In [160]:
totalbig = datatrainbig['workclass'].count()+3 #Tambah satu buat di laplace smoothing
totalsmall = datatrainsmall['workclass'].count()+3
d = {'big':[totalbig],'small':[totalsmall]}
totalbigsmall = pd.DataFrame(data=d)

youngbig = (datatrainbig['age'][datatrainbig['age'] == 'young'].count()+1)
adultbig = (datatrainbig['age'][datatrainbig['age'] == 'adult'].count()+1)
oldbig = (datatrainbig['age'][datatrainbig['age'] == 'old'].count()+1)
youngsmall = (datatrainsmall['age'][datatrainsmall['age'] == 'young'].count()+1)
adultsmall = (datatrainsmall['age'][datatrainsmall['age'] == 'adult'].count()+1)
oldsmall = (datatrainsmall['age'][datatrainsmall['age'] == 'old'].count()+1)
d = {'': ["young", "adult","old"], 'big': [youngbig, adultbig,oldbig], 'small': [youngsmall , adultsmall, oldsmall]}
umur = pd.DataFrame(data=d)

Privatebig = (datatrainbig['workclass'][datatrainbig['workclass'] == 'Private'].count()+1)
Selfbig = (datatrainbig['workclass'][datatrainbig['workclass'] == 'Self-emp-not-inc'].count()+1)
Localbig = (datatrainbig['workclass'][datatrainbig['workclass'] == 'Local-gov'].count()+1)
Privatesmall = (datatrainsmall['workclass'][datatrainsmall['workclass'] == 'Private'].count()+1)
Selfsmall = (datatrainsmall['workclass'][datatrainsmall['workclass'] == 'Self-emp-not-inc'].count()+1)
Localsmall = (datatrainsmall['workclass'][datatrainsmall['workclass'] == 'Local-gov'].count()+1)
d = {'': ["Private", "Self-emp-not-inc","Local-gov"], 'big': [Privatebig, Selfbig,Localbig], 'small': [Privatesmall , Selfsmall, Localsmall]}
workclass = pd.DataFrame(data=d)

Somebig = (datatrainbig['education'][datatrainbig['education'] == 'Some-college'].count()+1)
Bachelorsbig = (datatrainbig['education'][datatrainbig['education'] == 'Bachelors'].count()+1)
HSbig = (datatrainbig['education'][datatrainbig['education'] == 'HS-grad'].count()+1)
Somesmall = (datatrainsmall['education'][datatrainsmall['education'] == 'Some-college'].count()+1)
Bachelorssmall = (datatrainsmall['education'][datatrainsmall['education'] == 'Bachelors'].count()+1)
HSsmall = (datatrainsmall['education'][datatrainsmall['education'] == 'HS-grad'].count()+1)
d = {'': ["Some-college", "Bachelors","HS-grad"], 'big': [Somebig, Bachelorsbig,HSbig], 'small': [Somesmall , Bachelorssmall, HSsmall]}
education = pd.DataFrame(data=d)

Marriedbig = (datatrainbig['marital-status'][datatrainbig['marital-status'] == 'Married-civ-spouse'].count()+1)
Divorcedbig = (datatrainbig['marital-status'][datatrainbig['marital-status'] == 'Divorced'].count()+1)
Neverbig = (datatrainbig['marital-status'][datatrainbig['marital-status'] == 'Never-married'].count()+1)
Marriedsmall = (datatrainsmall['marital-status'][datatrainsmall['marital-status'] == 'Married-civ-spouse'].count()+1)
Divorcedsmall = (datatrainsmall['marital-status'][datatrainsmall['marital-status'] == 'Divorced'].count()+1)
Neversmall = (datatrainsmall['marital-status'][datatrainsmall['marital-status'] == 'Never-married'].count()+1)
d = {'': ["Married-civ-spouse", "Divorced","Never-married"], 'big': [Somebig, Bachelorsbig,HSbig], 'small': [Somesmall , Bachelorssmall, HSsmall]}
marital = pd.DataFrame(data=d)

Craftbig = (datatrainbig['occupation'][datatrainbig['occupation'] == 'Craft-repair'].count()+1)
Execbig = (datatrainbig['occupation'][datatrainbig['occupation'] == 'Exec-managerial'].count()+1)
Profbig = (datatrainbig['occupation'][datatrainbig['occupation'] == 'Prof-specialty'].count()+1)
Craftsmall = (datatrainsmall['occupation'][datatrainsmall['occupation'] == 'Craft-repair'].count()+1)
Execsmall = (datatrainsmall['occupation'][datatrainsmall['occupation'] == 'Exec-managerial'].count()+1)
Profsmall = (datatrainsmall['occupation'][datatrainsmall['occupation'] == 'Prof-specialty'].count()+1)
d = {'': ["Craft-repair", "Exec-managerial","Prof-specialty"], 'big': [Craftbig, Execbig,Profbig], 'small': [Craftsmall , Execsmall, Profsmall]}
occupation = pd.DataFrame(data=d)

Husbandbig = (datatrainbig['relationship'][datatrainbig['relationship'] == 'Husband'].count()+1)
Notbig = (datatrainbig['relationship'][datatrainbig['relationship'] == 'Not-in-family'].count()+1)
Ownbig = (datatrainbig['relationship'][datatrainbig['relationship'] == 'Own-child'].count()+1)
Husbandsmall = (datatrainsmall['relationship'][datatrainsmall['relationship'] == 'Husband'].count()+1)
Notsmall = (datatrainsmall['relationship'][datatrainsmall['relationship'] == 'Not-in-family'].count()+1)
Ownsmall = (datatrainsmall['relationship'][datatrainsmall['relationship'] == 'Own-child'].count()+1)
d = {'': ["Husband", "Not-in-family","Own-child"], 'big': [Husbandbig, Notbig,Ownbig], 'small': [Husbandsmall , Notsmall, Ownsmall]}
relationship = pd.DataFrame(data=d)

normalbig = (datatrainbig['hours-per-week'][datatrainbig['hours-per-week'] == 'normal'].count()+1)
lowbig = (datatrainbig['hours-per-week'][datatrainbig['hours-per-week'] == 'low'].count()+1)
manybig = (datatrainbig['hours-per-week'][datatrainbig['hours-per-week'] == 'many'].count()+1)
normalsmall = (datatrainsmall['hours-per-week'][datatrainsmall['hours-per-week'] == 'normal'].count()+1)
lowsmall = (datatrainsmall['hours-per-week'][datatrainsmall['hours-per-week'] == 'low'].count()+1)
manysmall = (datatrainsmall['hours-per-week'][datatrainsmall['hours-per-week'] == 'many'].count()+1)
d = {'': ["normal", "low","many"], 'big': [normalbig, lowbig,manybig], 'small': [normalsmall , lowsmall,manysmall]}
hours = pd.DataFrame(data=d)

# normal = datatrain['hours-per-week'][data['hours-per-week'] == 'normal'].count()/total
# low = datatrain['hours-per-week'][data['hours-per-week'] == 'low'].count()/total

# lebih = datatrain['income'][data['income'] == '<=50K'].count()
# kurang = datatrain['income'][data['income'] == '>50K'].count()

# data_means = np.var([lebih,kurang])

print(umur)
print() 
print(workclass)
print() 
print(education)
print() 
print(marital)
print() 
print(occupation)
print() 
print(relationship)
print() 
print(hours) 
print() 
print(totalbigsmall) 

          big  small
0  young   44     21
1  adult   37     20
2    old    2      2

                     big  small
0           Private   72     33
1  Self-emp-not-inc    4      8
2         Local-gov    7      2

                 big  small
0  Some-college   20     17
1     Bachelors   45      8
2       HS-grad   18     18

                       big  small
0  Married-civ-spouse   20     17
1            Divorced   45      8
2       Never-married   18     18

                    big  small
0     Craft-repair   26     22
1  Exec-managerial   32     16
2   Prof-specialty   25      5

                  big  small
0        Husband   76     20
1  Not-in-family    5     15
2      Own-child    2      8

           big  small
0  normal   77     34
1     low    4      6
2    many    2      3

   big  small
0   83     43


## BUAT PREDIKSI

In [161]:
def prediksi(datatest):
  hasiltest = []
  nilaibesar = 1
  nilaikecil = 1
  for i, row1 in datatest.iterrows():
    for j, row2 in umur.iterrows():
      if row1['age'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in workclass.iterrows():
      if row1['workclass'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in education.iterrows():
      if row1['education'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in marital.iterrows():
      if row1['marital-status'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in occupation.iterrows():
      if row1['occupation'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in relationship.iterrows():
      if row1['relationship'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in hours.iterrows():
      if row1['hours-per-week'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    totaldata = totalbig+totalsmall
    nilaibesar *= totalbig/(totalbig**7)/totaldata
    nilaikecil *= totalsmall/(totalsmall**7)/totaldata
    if nilaibesar > nilaikecil:
      hasiltest.append('>50K')
    if nilaibesar <= nilaikecil:
      hasiltest.append('<=50K')
  return hasiltest
a = prediksi(datatest)
a

['<=50K',
 '<=50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K']

In [162]:
datatest

,id,age,workclass,education,marital-status,occupation,relationship,hours-per-week,income
0,3019,adult,Private,Some-college,Never-married,Exec-managerial,Not-in-family,normal,<=50K
1,6045,adult,Self-emp-not-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,normal,>50K
2,27988,adult,Local-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,normal,>50K
3,13128,adult,Local-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,normal,>50K
4,29274,young,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,normal,>50K
5,25885,young,Self-emp-not-inc,Bachelors,Never-married,Prof-specialty,Not-in-family,normal,>50K
6,19644,adult,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,normal,>50K
7,6994,adult,Local-gov,HS-grad,Married-civ-spouse,Exec-managerial,Husband,normal,>50K
8,9847,young,Self-emp-not-inc,Some-college,Married-civ-spouse,Craft-repair,Husband,normal,>50K
9,24113,young,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,normal,>50K


In [163]:
predict = pd.read_csv('NaiveBayes/TestsetTugas1ML.csv',skipinitialspace=True)
prediksi(predict)


['<=50K',
 '<=50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '<=50K',
 '<=50K',
 '<=50K',
 '>50K',
 '>50K',
 '<=50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K']